In [18]:
import random
import numpy as np
from scipy.stats import cauchy
import copy
from scipy.optimize import fmin_l_bfgs_b
from decimal import Decimal
import scipy.io as scio
from tqdm import tqdm
import hashlib

In [19]:
random.seed(2)
np.random.seed(2)

In [20]:
import numpy as np
import scipy.io as scio
import hashlib
from functools import lru_cache 
class GNBG:
    def __init__(self, i):
        # get file path './fi.mat/' 
        if(i < 1 or i > 24):
            raise ValueError("problem index must be in [1, 24]")
        self.idx = i
        self.file_path = './gnbg_func/f' + str(i) + '.mat'
        self.GNBG = self.get_GNBG_from_path(self.file_path)
        self.LB = self.GNBG['MinCoordinate'][0][0][0][0]
        self.UB = self.GNBG['MaxCoordinate'][0][0][0][0]
        self.dim = self.GNBG['Dimension'][0][0][0][0]
        self.num_com = self.GNBG['o'][0][0][0][0]
        self.opt_value = self.GNBG['OptimumValue'][0][0][0][0]
        self.opt_position = self.GNBG['OptimumPosition'][0][0]
        self.best_val = np.inf
        self.FE = 0
        self.aceps = np.inf
        self.fitness_map = {}
        self.max_FE = 500000 if i <= 15 else 1000000
    def get_GNBG_from_path(self, file_path):
        data = scio.loadmat(file_path)
        GNBG = data['GNBG']
        # print("ID:", self.idx)
        # print("Optimal value: ", GNBG['OptimumValue'][0][0][0][0])
        # print("Num components: ", GNBG['o'][0][0][0][0])
        # print("Dimension: ", GNBG['Dimension'][0][0][0][0])
        # print("LB: ", GNBG['MinCoordinate'][0][0][0][0])
        # print("UB: ", GNBG['MaxCoordinate'][0][0][0][0])
        return GNBG
    def check(self):
        ind = self.fitness_of_ind(self.opt_position)
        val = self.opt_value
        print("Fitness of our code", ind)
        print("Fitness of the author", val)
    def fitness_of_pop(self, X):
        '''
        Input : A matrix of size (N,d), 
        N: number of individuals, d: number of dimensions, 
        kth row (1<=k<=N) represents kth individual vector.
        Output: a matrix (N,1) representing the fitness of N individuals.

        '''
        ### read the MATLAB file and call a few indices to get the same shape as the original competition file
        num_com = self.GNBG['o'][0][0][0][0]
        dim = self.GNBG['Dimension'][0][0][0][0]
        min_pos = self.GNBG['Component_MinimumPosition'][0][0].reshape(num_com, dim)
        rot_mat = self.GNBG['RotationMatrix'][0][0].reshape(dim,dim,num_com)
        sigma = self.GNBG['ComponentSigma'][0,0].reshape(num_com,1)
        hehe = self.GNBG['Component_H'][0,0].reshape(num_com,dim)
        mu = self.GNBG['Mu'][0][0].reshape(num_com,2)
        omega = self.GNBG['Omega'][0][0].reshape(num_com,4)
        lamb = self.GNBG['lambda'][0][0].reshape(num_com,1)
        SolutionNumber, _ = X.shape
        result = np.empty((SolutionNumber, 1))
        ### Converts the input vector format and returns the fitness list of individuals
        for jj in range(SolutionNumber):
            self.FE = self.FE + 1
            x = X[jj, :].reshape(-1,1)
            f = np.empty((1, num_com))
            
            for k in range(num_com):
                inp = x - min_pos[k,:].reshape(-1,1)
                a = Transform( inp.T  @  rot_mat[:,:,k] , mu[k,:], omega[k,:])
                b = Transform(rot_mat[:,:,k] @ inp, mu[k,:], omega[k,:] )
                f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]

            result[jj] = np.min(f)
            
            # if GNBG['FE'] > GNBG['MaxEvals']:
            #     return result, GNBG
            
            # GNBG['FE'] += 1
            # GNBG['FEhistory'][GNBG['FE']] = result[jj]
            
            # if GNBG['BestFoundResult'] > result[jj]:
            #     GNBG['BestFoundResult'] = result[jj]
                
            # if (abs(GNBG['FEhistory'][GNBG['FE']] - GNBG['OptimumValue']) < GNBG['AcceptanceThreshold'] and
            #         np.isinf(GNBG['AcceptanceReachPoint'])):
            #     GNBG['AcceptanceReachPoint'] = GNBG['FE']
        
        return result

    def fitness_of_ind(self,X):
        # genes_hash = hashlib.sha256(X.tobytes()).hexdigest()
        # if genes_hash in self.fitness_map:
        #     print("Hit")
        #     return self.fitness_map[genes_hash]
        
        num_com = self.GNBG['o'][0][0][0][0]
        dim = self.GNBG['Dimension'][0][0][0][0]
        min_pos = self.GNBG['Component_MinimumPosition'][0][0].reshape(num_com, dim)
        rot_mat = self.GNBG['RotationMatrix'][0][0].reshape(dim,dim,num_com)
        sigma = self.GNBG['ComponentSigma'][0,0].reshape(num_com,1)
        hehe = self.GNBG['Component_H'][0,0].reshape(num_com,dim)
        mu = self.GNBG['Mu'][0][0].reshape(num_com,2)
        omega = self.GNBG['Omega'][0][0].reshape(num_com,4)
        lamb = self.GNBG['lambda'][0][0].reshape(num_com,1)
        ### Converts the input vector format and returns the fitness list of individuals
        self.FE = self.FE + 1
        x = X.reshape(-1,1)
        f = np.empty((1, num_com))
        for k in range(num_com):
            inp = x - min_pos[k, :].reshape(-1,1)
            a = Transform( inp.T  @  rot_mat[:,:,k],mu[k, :],omega[k,:])
            b = Transform(rot_mat[:,:,k] @ inp, mu[k, :], omega[k, :] )
            f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]

        
        result = np.min(f)
        self.best_val = min(self.best_val, result)
        # add result and corresponding genes to the map
        # genes_hash = hashlib.sha256(X.tobytes()).hexdigest()
        # self.fitness_map[genes_hash] = result
        if abs(result - self.opt_value) < 1e-8 and np.isinf(self.aceps):
            self.aceps = self.FE
            
            # if GNBG['FE'] > GNBG['MaxEvals']:
            #     return result, GNBG
            
            # GNBG['FE'] += 1
            # GNBG['FEhistory'][GNBG['FE']] = result[jj]
            
            # if GNBG['BestFoundResult'] > result[jj]:
            #     GNBG['BestFoundResult'] = result[jj]
                
            # if (abs(GNBG['FEhistory'][GNBG['FE']] - GNBG['OptimumValue']) < GNBG['AcceptanceThreshold'] and
            #         np.isinf(GNBG['AcceptanceReachPoint'])):
            #     GNBG['AcceptanceReachPoint'] = GNBG['FE']
        
        return result
        
    def fitness_mfea(self, X):
        # ##########################################################################
        # Due to float preicision, this function may not return the right value  #
        ##########################################################################
        return self.fitness_of_ind(self.LB + np.array(X)*(self.UB-self.LB))

def Transform(X, Alpha, Beta):
    Y = X.copy()
    tmp = (X > 0)
    Y[tmp] = np.log(X[tmp])
    Y[tmp] = np.exp(Y[tmp] + Alpha[0] * (np.sin(Beta[0] * Y[tmp]) + np.sin(Beta[1] * Y[tmp])))
    
    tmp = (X < 0)
    Y[tmp] = np.log(-X[tmp])
    Y[tmp] = -np.exp(Y[tmp] + Alpha[1] * (np.sin(Beta[2] * Y[tmp]) + np.sin(Beta[3] * Y[tmp])))
    
    return Y




In [21]:
class jSOError(Exception):
    """exception class for number of generations set two low."""
    def __init__(self, message):
        super().__init__(message)
class jSO:
    def __init__(self, prob, init_F=0.5, init_CR=0.8, H=5, best_rate = 0.25, archive_rate = 2.6,
    G_max = 2500):
    # if you see jsoError and it leads you here, 
    # change G_max to a bigger number (like +100, +200) until the error is gone, but not too big
        self.prob = prob
        self.H = H
        self.effective = None
        self.G_max = G_max
        self.G = 0
        self.mem_cr = [init_CR] * self.H
        self.mem_f = [init_F] * self.H
        self.s_cr = []
        self.s_f = []
        self.diff_f = []
        self.mem_pos = 0
        self.archive_rate = archive_rate
        self.archive = []
        self.p_max = best_rate
        self.p_min = best_rate/2
        self.p = best_rate
    def generateFCR(self, m_f, m_cr):
        if m_cr < 0:
            cr = 0
        else:
            cr = np.random.normal(loc=m_cr, scale=0.1)
            cr = np.clip(cr, 0, 1)
        while True:
            f = cauchy.rvs(loc = m_f, scale=0.1, size=1 )[0]
            if f>=0:
                break
        if self.G < 0.25*self.G_max:
            f = min(f, 0.7)
            cr = max(cr, 0.5)
        elif self.G < 0.5*self.G_max:
            f = min(f, 0.8)
            cr = max(cr, 0.25)
        elif self.G < 0.75*self.G_max:
            f = min(f, 0.9)
        
        return min(f,1), min(cr,1)
    def cur_to_pbest_w_1_bin(self, pool, ind, cr, f, best):
        pbest = best[random.randint(0, len(best) - 1)]
        rand_idx = np.random.choice(len(pool), 2, replace=False)
        ind_ran1 = pool[rand_idx[0]]
        ind_ran2 = random.sample(pool+self.archive, 1)[0]
        u = (np.random.uniform(0, 1, size=(len(ind.genes)) ) <= cr)
        u[np.random.choice(len(ind.genes))] = True
        if self.prob.FE < 0.2*self.prob.max_FE:
            coefficent = 0.7
        elif self.prob.FE < 0.4*self.prob.max_FE:
            coefficent = 0.8
        else:
            coefficent = 1.2
        new_genes = np.where(u,
            ind.genes + f*coefficent*(pbest.genes - ind.genes)+f*(ind_ran1.genes-ind_ran2.genes),
            ind.genes
        )
        bounds = ind.bounds
        for j in range(len(new_genes)):
                if new_genes[j] > bounds[1]:
                    new_genes[j] = (bounds[1] + ind.genes[j]) / 2
                elif new_genes[j] < bounds[0]:
                    new_genes[j] = (bounds[0] + ind.genes[j]) / 2
        new_ind = Individual(ind.MAX_NVARS, bounds)
        new_ind.genes = new_genes
        new_ind.cal_fitness(self.prob)
        return new_ind
    def addarchive(self, ind, pop_size):
        caution_size = int(self.archive_rate * pop_size)
        while len(self.archive) > caution_size:
            r = random.randint(0, len(self.archive) - 1)
            self.archive.pop(r)
        self.archive.append(ind)
    def search(self, pool):
        if self.G >= self.G_max and self.prob.FE < self.prob.max_FE:
            raise jSOError(f'Number of FEs left is {self.prob.max_FE - self.prob.FE}, but generation ended: Current G : {self.G}, max G : {self.G_max}')
        self.G += 1
        pool.sort(key = lambda x : x.fitness)
        best = pool[: max( int(self.p*len(pool)), 2 )]
        new_pool = []
        random.shuffle(pool)
        for ind in pool:
            r = random.randint(0, self.H - 1)
            if r==self.H - 1:
                self.mem_cr[r] = 0.9
                self.mem_f[r] = 0.9
            f, cr = self.generateFCR(self.mem_f[r], self.mem_cr[r])
            new_ind = self.cur_to_pbest_w_1_bin(pool, ind, cr, f, best)
            if new_ind.fitness < ind.fitness:
                self.diff_f.append(new_ind.fitness - ind.fitness)
                self.s_cr.append(cr)
                self.s_f.append(f)
                new_pool.append(new_ind)
                self.addarchive(ind, len(pool))
            else:
                new_pool.append(ind)
        self.p = (self.p_max - self.p_min) / self.prob.max_FE * self.prob.FE + self.p_min
        self.updateMemory()
        return new_pool
    def updateMemoryCR(self, diff_f, s_cr):
        diff_f = np.array(diff_f)
        s_cr = np.array(s_cr)
        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tu = sum(weight * s_cr * s_cr)
        mau = sum(weight * s_cr)
        return tu/mau
    def updateMemoryF(self, diff_f, s_f):
        diff_f = np.array(diff_f)
        s_f = np.array(s_f)
        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tu = sum(weight * s_f * s_f)
        mau = sum(weight * s_f)
        return tu/mau
    def updateMemory(self):
        if len(self.s_cr) > 0:
            if self.mem_cr[self.mem_pos] == -1 or max(self.s_cr)==0:
                self.mem_cr[self.mem_pos] = -1
            else:
                self.mem_cr[self.mem_pos] = self.updateMemoryCR(self.diff_f, self.s_cr)
            self.mem_f[self.mem_pos] = self.updateMemoryF(self.diff_f, self.s_f)
            self.mem_pos = (self.mem_pos + 1) % self.H
            self.s_cr = []
            self.s_f = []
            self.diff_f = []
        else:
            # do nothing
            pass

        

In [22]:
class Parameter:
    reps = 30
    MAX_FEs = 60000
    numRecords = 1000
    SUBPOPULATION = 100
    SIZE_POPULATION = 1000
    MAX_GENERATION = 1000

    mum = 5
    mu = 2

    rmp = 0.7
    pc = 0.8
    pm = 0.3

    num_fitness = 0
    countFitness = None
    o_rmp = None  
    FEs = None
    FEl = None

    @staticmethod
    def initialize_o_rmp():

        pass

In [23]:
class Individual:
    def __init__(self, MAX_NVARS, bounds):
        self.MAX_NVARS = MAX_NVARS
        self.genes = np.zeros(self.MAX_NVARS)
        self.fitness = np.inf
        self.bounds = bounds

    def init(self):
        self.genes = np.random.uniform(self.bounds[0], self.bounds[1], self.MAX_NVARS)

    def fcost(self):
        return self.fitness
    def cal_fitness(self, prob):
        self.fitness = prob.fitness_of_ind(self.genes)

In [24]:
class Population:
    def __init__(self, SIZEPOP, SIZEGENES, prob, **kwargs):
        self.SIZEPOP = SIZEPOP
        #self.rand = rand
        self.SIZE_GENES = SIZEGENES
        self.pop = []
        self.prob = prob
        self.das_prob = None
    def init(self):
        for i in range(self.SIZEPOP):
            ind = Individual(self.SIZE_GENES, [-100,100])
            ind.init()
            ind.cal_fitness(self.prob)
            self.pop.append(ind)
    def get_subpops(self):
        return [self.pop]
    
    def get_result(self):
        result = np.inf
        for ind in self.pop:
            if ind.fitness < result:
                result = ind.fitness
        return result
    
    # def calculateD(self):
    #     subpops = self.get_subpops()
    #     D = np.zeros(len(self.prob))
    #     maxFit = np.zeros(len(self.prob))
    #     minFit = np.zeros(len(self.prob))
        
    #     for i in range(len(subpops)):
    #         subpops[i].sort(key=lambda ind: ind.fitness[i])
    #         maxFit[i] = subpops[i][-1].fitness[i]
    #         minFit[i] = subpops[i][0].fitness[i]
    #         sum_fitness = sum([ind.fitness[i] for ind in subpops[i]])
    #         for idx, ind in enumerate(subpops[i]):
    #             if(idx !=0):
    #                 wx = 1 - ind.fitness[i]/sum_fitness
    #                 distance = np.linalg.norm(ind.genes - subpops[i][0].genes)
    #                 D[i] += wx * distance
    #     return D, maxFit, minFit


    # def update_scalar_fitness(self):
    #     pop = self.pop
    #     for i, task in enumerate(self.prob):
    #         pop.sort(key = lambda ind: ind.fitness[i])
    #         for j in range(len(pop)):
    #             pop[j].rank[i]=j
    #     for ind in pop:
    #         _min = float('inf')
    #         _task = 0
    #         for task in range(1, len(self.prob)+1):
    #             if _min > ind.rank[task-1]:
    #                 _min = ind.rank[task-1]
    #                 _task = task 
    #             elif _min == ind.rank[task-1]:
    #                 if(random.random()<0.5):
    #                     _task = task
    #         ind.skill_factor = _task
    #         ind.scalar_fitness = 1.0/(_min + 1)
    def crossover(self, parent1, parent2):
        assert parent1.bounds == parent2.bounds
        bounds = parent1.bounds
        offspring = []
        p1 = parent1.genes
        p2 = parent2.genes
        p1 = (p1 - bounds[0] ) / (bounds[1] - bounds[0])
        p2 = (p2 - bounds[0] ) / (bounds[1] - bounds[0])
        D = p1.shape[0]
        cf = np.empty([D])
        u = np.random.rand(D)        

        cf[u <= 0.5] = np.power((2 * u[u <= 0.5]), (1 / (Parameter.mu + 1)))
        cf[u > 0.5] = np.power((2 * (1 - u[u > 0.5])), (-1 / (Parameter.mu + 1)))

        c1 = 0.5 * ((1 + cf) * p1 + (1 - cf) * p2)
        c2 = 0.5 * ((1 + cf) * p2 + (1 - cf) * p1)

        c1 = np.clip(c1, 0, 1)
        c2 = np.clip(c2, 0, 1)
        swap = True
        if swap:
            idx_swap = np.where(np.random.rand(self.SIZE_GENES) < 0.5)[0]
            c1[idx_swap], c2[idx_swap] = c2[idx_swap], c1[idx_swap]
      
        c1 = c1*(bounds[1] - bounds[0]) + bounds[0]
        c2 = c2*(bounds[1] - bounds[0]) + bounds[0]
        child1 = Individual(self.SIZE_GENES,  bounds = bounds)
        child2 = Individual(self.SIZE_GENES, bounds = bounds)
        child1.genes = c1 
        child2.genes = c2
        offspring.extend([child1, child2])
        return offspring
    #  for i in range(self.SIZE_GENES):
    #     v = 0.5 * ((1 + cf[i]) * parent1.genes[i] + (1 - cf[i]) * parent2.genes[i])
    #     v2 = 0.5 * ((1 - cf[i]) * parent1.genes[i] + (1 + cf[i]) * parent2.genes[i])

    #     child1.genes[i] = min(1, max(0, v))
    #     child2.genes[i] = min(1, max(0, v2))

    #  off_spring.extend([child1, child2])
    #  return off_spring


    def selection(self, pop_size):
        self.pop.sort(key = lambda ind: ind.scalar_fitness, reverse=True)
        size = len(self.pop)
        if len(self.pop) > pop_size:
            del self.pop[pop_size:size]
    
    def selection_EMEBI(self, pop_size):
        ################## PYMSOO IMPLEMENTATION #######################
        # subpops = self.get_subpops()
        # new_population = []
        # for i in range(len(inds_tasks)):
        #     subpop = subpops[i]
        #     subpop_factorial_rank = np.argsort(np.argsort([ind.fitness[i] for ind in subpop]))
        #     subpop_scalarfit = 1/(subpop_factorial_rank + 1) 

        #     Ni = min(inds_tasks[i], len(subpop))
    
        #     idx_selected = np.where(subpop_scalarfit > 1/(Ni + 1))[0].tolist()
        #     # remain_idx = np.where(subpop_scalarfit < 1/(Netil))[0].tolist()
        #     # idx_random = np.random.choice(remain_idx, size= (int(Ni - Netil), )).tolist()
        #     # idx_selected += idx_random
        #     new_population.extend([subpop[i] for i in idx_selected])
        # self.pop = new_population

         ##################   ANOTHER WAY TO IMPLEMENTATION ########################


        new_population = []
        N = min(pop_size, len(self.pop))
        self.pop.sort(key = lambda ind: ind.fitness)
        new_population.extend([self.pop[i] for i in range(N)])
        self.pop = new_population
        # for i in range(len(inds_tasks)):
        #     subpop = subpops[i]
        #     Ni = min(inds_tasks[i], len(subpop))
        #     subpop.sort(key = lambda ind : ind.fitness[i])
        #     new_population.extend([subpop[i] for i in range(Ni)])
        # self.pop = new_population


    # def variable_swap(self, p1, p2):
    #     ind1 = p1
    #     ind2 = p2

    #     for i in range(1, self.SIZE_GENES + 1):
    #         if random.random() > 0.5:
    #             temp1 = p1.genes[i - 1]
    #             temp2 = p2.genes[i - 1]
    #             ind1.genes[i - 1] = temp2
    #             ind2.genes[i - 1] = temp1
    # def gauss_mutation(self, ind):
    #     p = 0.01
    #     for i in range(len(ind.genes)):
    #         if random.random() < 1.0 / len(ind.genes):
    #             t = ind.genes[i] + random.gauss(0, 1)
    #             if t > 1:
    #                 t = ind.genes[i] + random.random() * (1 - ind.genes[i])
    #             elif t < 0:
    #                 t = random.random() * ind.genes[i]

    #             ind.genes[i] = t


In [25]:
class SubPop():
    def __init__(self, pool, prob, searcher=None):
        self.prob= prob
        self.pool = pool     
        self.searcher = searcher 
        self.fitness_improv = 0
        self.consume_fes = 0
        self.dim = 30
        # self.task = prob[self.skill_factor - 1]
        self.scale = 0.1
    def search(self):
        return self.searcher.search(self.pool)

In [26]:
class Mutation:
    def __init__(self, prob, survival_rate = None):
        self.effective = None
        self.survival_rate = survival_rate if survival_rate is not None else 0
        self.prob = prob
        self.key = random.random()
    def mutation_ind(self, parent):
        p = parent.genes
        bounds = parent.bounds
        p = (p - bounds[0]) / (bounds[1] - bounds[0])
        mp = float(1. / p.shape[0])
        u = np.random.uniform(size=[p.shape[0]])
        r = np.random.uniform(size=[p.shape[0]])             
        tmp = np.copy(p)
        for i in range(p.shape[0]):
            if r[i] < mp:
                if u[i] < 0.5:
                    delta = (2*u[i]) ** (1/(1+Parameter.mum)) - 1
                    tmp[i] = p[i] + delta * p[i]
                else:
                    delta = 1 - (2 * (1 - u[i])) ** (1/(1+Parameter.mum))
                    tmp[i] = p[i] + delta * (1 - p[i])
        tmp = np.clip(tmp, 0, 1)
        tmp = tmp * (bounds[1] - bounds[0]) + bounds[0]
        ind = Individual(len(parent.genes), bounds)
        ind.genes = tmp
        ind.cal_fitness(self.prob)
        return ind
    def search(self, pool):
        accumulate_diff = 0
        new_pool = []
        old_FEs = self.prob.FE
        for ind in pool:
            new_ind = self.mutation_ind(ind)
            diff = ind.fitness - new_ind.fitness
            if diff > 0:
                new_pool.append(new_ind)
                accumulate_diff += diff
            else:
                new_pool.append(ind)
        new_FEs = self.prob.FE
        self.effective = accumulate_diff / (new_FEs - old_FEs)
        
        return new_pool
class MutationLS:
    def __init__(self, prob):
        self.prob = prob
        self.effective = float('inf')
    def search(self, ind, maxeval):
        # Mutation only use 1 function evaluation
        p = ind.genes
        bounds = ind.bounds
        p = (p - bounds[0]) / (bounds[1] - bounds[0])
        mp = float(1. / p.shape[0])
        u = np.random.uniform(size=[p.shape[0]])
        r = np.random.uniform(size=[p.shape[0]])
        tmp = np.copy(p)
        for i in range(p.shape[0]):
            if r[i] < mp:
                if u[i] < 0.5:
                    delta = (2 * u[i]) ** (1 / (1 + Parameter.mum)) - 1
                    tmp[i] = p[i] + delta * p[i]
                else:
                    delta = 1 - (2 * (1 - u[i])) ** (1 / (1 + Parameter.mum))
                    tmp[i] = p[i] + delta * (1 - p[i])
        tmp = np.clip(tmp, 0, 1)
        tmp = tmp * (bounds[1] - bounds[0]) + bounds[0]
        new_ind = Individual(len(ind.genes), bounds)
        new_ind.genes = tmp
        new_ind.cal_fitness(self.prob)
        if new_ind.fitness < ind.fitness:
            self.effective = (ind.fitness - new_ind.fitness) / abs(ind.fitness)
            return new_ind
        else:
            self.effective = 0
            return ind

In [27]:
class LearningPhase():
    M = 2   #number of operators
    def __init__(self, is_start, prob) -> None:
        self.prob = prob
        self.start = is_start
        self.searcher = [ IMODE(0.5, 0.5, prob)]
    def evolve(self, subpop, divisionRate = 0.4) :
        # split subpop into sub_pools which len(sub_pools)==len(self.searcher), 
        # each sub_pool is assigned an operator
        random.shuffle(subpop)
        new_pool = []
        sub_pools = []
        if divisionRate > 1/len(self.searcher):
            raise ValueError(f"divisionRate must be less than 1/num_searcher, but got {divisionRate} and {1/len(self.searcher)} respectively.")
        m = len(self.searcher)
        n = int ((1 - (m-1)* divisionRate) * len(subpop))
        # print("effective: ", [search.effective for search in self.searcher])
        
        for i in range(len(self.searcher)):
            if(i==0):
                sub_pools.append(SubPop(subpop[:n], self.prob))
            else:
                sub_pools.append(SubPop(subpop[int(n + (i-1)*divisionRate*len(subpop)):int(n + i*divisionRate*len(subpop))], self.prob))
        assert len(sub_pools) == len(self.searcher)
        

        if self.start:
            opcode = list(range(len(self.searcher)))
            random.shuffle(opcode)
            for idx, subpop in enumerate(sub_pools):
                subpop.searcher = self.searcher[opcode[idx]]
           
        else:
            opcode = np.argsort([search.effective for search in self.searcher])[::-1]
            for i, idx in enumerate(opcode):
                sub_pools[i].searcher = self.searcher[idx]
    
        for i in range(len(self.searcher)):
            pool = sub_pools[i].search()

            new_pool.extend(pool)
        return new_pool

class LearningPhaseILS():
    def __init__(self, prob):
        self.prob = prob
        self.searcher = [MTSLS1(prob), LBFGSB(prob), MutationLS(prob)]
        self.explorer = IMODE(0.5, 0.5, prob)
        self.stay = 0
        self.use_restart = True
        self.grad_threshold = 30

    def evolve(self, subpop, DE_evals, LS_evals):
        ####### Explorer part (already implemented in Phase 2) ##########
        max_DE = self.prob.FE + DE_evals
        while self.prob.FE < max_DE:
            subpop = self.explorer.search(subpop)
        ################################################################
        # subpop.sort(key = lambda ind: ind.fitness)
        # if self.searcher[0].effective > self.searcher[1].effective:
        #     this_iteration_searcher = self.searcher[0]
        # if self.searcher[0].effective < self.searcher[1].effective:
        #     this_iteration_searcher = self.searcher[1]
        # if self.searcher[0].effective == self.searcher[1].effective:
        #     this_iteration_searcher = random.choice(self.searcher)
        self.searcher.sort(key = lambda search: search.effective, reverse=True)
        highest_effective = self.searcher[0].effective
        top_searchers = [s for s in self.searcher if s.effective == highest_effective]
        this_iteration_searcher = random.choice(top_searchers)
        # print(f'This iter use {this_iteration_searcher.__class__.__name__}, effective: {this_iteration_searcher.effective}')
        subpop[0] = this_iteration_searcher.search(subpop[0], LS_evals)
        this_fitness = subpop[0].fitness
         ############ One more L-BFGS-B with random starting point ######################
        if self.use_restart:
            ind = copy.deepcopy(subpop[0])
            sol2, fit2, info2 = fmin_l_bfgs_b(self.prob.fitness_of_ind, np.random.uniform(-100, 100, len(ind.genes)), approx_grad=True, bounds=[(-100,100)]*len(ind.genes), maxfun=LS_evals, factr=10)
            if fit2 < this_fitness:
                new_ind = Individual(ind.MAX_NVARS, ind.bounds)
                new_ind.genes = sol2
                new_ind.fitness = fit2
                subpop[0] = new_ind
         ##### Check if we use restart next time or not #########
            if np.linalg.norm(info2["grad"]) > self.grad_threshold:
                self.use_restart = False
            if fit2 > this_fitness:
                disimprove_ratio = (fit2 - this_fitness) / abs(this_fitness)
                if disimprove_ratio > 5:
                    self.use_restart = False
                # print(f'Disimprove ratio: {disimprove_ratio}')
        return subpop

In [28]:

class LearningPhaseILSVer2():
    def __init__(self, prob):
        self.prob = prob
        self.searcher = [MTSLS1(prob), LBFGSB2(prob), MutationLS(prob)]
        self.explorer = jSO(prob)
        self.stagnation_threshold = [40, 40, 40]
        self.stagnation = [0]*len(self.searcher)
        self.LS_useful = True
        # self.explorer = jSO(prob)
        self.stay = 0
        self.use_restart = True
        self.grad_threshold = 30
    def evolve(self, subpop, DE_evals, LS_evals):
        max_DE = self.prob.FE + DE_evals
        while self.prob.FE < max_DE:
            subpop = self.explorer.search(subpop)
        subpop.sort(key = lambda ind: ind.fitness)
        # calculate coefficent of local searchs:
        coefficent = [0]*len(self.searcher)
        sum_stagnation = sum(self.stagnation)
        for i in range(len(self.searcher)):
            if sum_stagnation != 0 and self.stagnation[i] <= self.stagnation_threshold[i]:
                coefficent[i] = 1 - (self.stagnation[i])/sum_stagnation
            elif sum_stagnation == 0:
                coefficent[i] = 1
            else:
                coefficent[i] = 0
        # if all local searchs exceed threshold, then we shut down local search
        if(sum(coefficent) == 0):
            self.LS_useful = False
        if self.LS_useful:
            selected_idx = random.choices(range(len(self.searcher)), weights=coefficent, k=1)[0]
            # print(f'Use {self.searcher[selected_idx].__class__.__name__} -- stagnation {self.stagnation[selected_idx]}')
            selected_local_search = self.searcher[selected_idx]
            new_ind = selected_local_search.search(subpop[0], LS_evals)
            assert new_ind.fitness <= subpop[0].fitness, f'New fitness after LS: {new_ind.fitness} > Old fitness: {subpop[0].fitness}'
            # if fitness doesn't improve, we incrase stagnation
            if new_ind.fitness == subpop[0].fitness:
                if selected_local_search.__class__.__name__ == 'MTSLS1':
                    self.stagnation[selected_idx] += 8
                elif selected_local_search.__class__.__name__ == 'LBFGSB2':
                    if np.linalg.norm(selected_local_search.grad) > self.grad_threshold:
                        self.stagnation[selected_idx] += 8
                    else:
                        self.stagnation[selected_idx] += 1
                else:
                    self.stagnation[selected_idx] += 1
            else:
                subpop[0] = new_ind
        return subpop
        
            
        
    

In [29]:
class IMODE:
    def __init__(self, init_F, init_CR, prob, survival_rate = None, H = 6, best_rate = 0.1, arc_rate = 2.6 ):
        self.prob = prob
        self.H = H
        self.effective = None
        self.survival_rate = survival_rate if survival_rate is not None else 0
        self.mem_cr = [init_CR] * self.H
        self.mem_f = [init_F] * self.H
        self.s_cr = []
        self.s_f = []
        self.diff_f = []
        self.archive = []
        self.ratio = [1/3 , 1/3, 1/3]
        self.mem_pos = 0
        self.arc_rate = arc_rate
        self.key = random.random()
    
    def generateFCR(self, m_f, m_cr):
        if m_cr == "terminal":
            cr = 0
        else:
            cr = np.random.normal(loc=m_cr, scale=0.1)
        if cr < 0:
            cr = 0
        if cr > 1:
            cr = 1
        while True:
            f = cauchy.rvs(loc = m_f, scale=0.1, size=1 )[0]
            if f>=0:
                break
        return min(f,1), min(cr,1)
    def cur_topbest(self, pool, ind, cr, f, best):
        pbest = best[random.randint(0, len(best) - 1)]
        rand_idx = np.random.choice(len(pool), 2, replace=False)
        ind_ran1, ind_ran2 = pool[rand_idx[0]], pool[rand_idx[1]]
        u = (np.random.uniform(0, 1, size=(len(ind.genes)) ) <= cr)
        u[np.random.choice(len(ind.genes))] = True
        new_genes = np.where(u,
            ind.genes + f * (pbest.genes - ind.genes + ind_ran1.genes - ind_ran2.genes),
            ind.genes
        )
        bounds = ind.bounds
        for j in range(len(new_genes)):
                if new_genes[j] > bounds[1]:
                    new_genes[j] = (bounds[1] + ind.genes[j]) / 2
                elif new_genes[j] < bounds[0]:
                    new_genes[j] = (bounds[0] + ind.genes[j]) / 2

        new_ind = Individual(ind.MAX_NVARS, bounds)
        new_ind.genes = new_genes
        new_ind.cal_fitness(self.prob)
        return new_ind
    def cur_topbestav(self, pool, ind, cr, f, best):
        pbest = best[random.randint(0, len(best) - 1)]
        ind_ran1, ind_ran2 = random.sample(pool + self.archive, 2)
        u = (np.random.uniform(0, 1, size=(len(ind.genes)) ) <= cr)
        u[np.random.choice(len(ind.genes))] = True
        new_genes = np.where(u,
            ind.genes + f * (pbest.genes - ind.genes + ind_ran1.genes - ind_ran2.genes),
            ind.genes
        )
        bounds = ind.bounds
        for j in range(len(new_genes)):
                if new_genes[j] > bounds[1]:
                    new_genes[j] = (bounds[1] + ind.genes[j]) / 2
                elif new_genes[j] < bounds[0]:
                    new_genes[j] = (bounds[0] + ind.genes[j]) / 2

        new_ind = Individual(ind.MAX_NVARS, bounds)
        new_ind.genes = new_genes
        new_ind.cal_fitness(self.prob)
        return new_ind


    def pbest_ind(self, pool, ind, cr, f, best):
        pbest = best[random.randint(0, len(best) - 1)]
        rand_idx = np.random.choice(len(pool), 2, replace=False)
        ind_ran1, ind_ran2 = pool[rand_idx[0]], pool[rand_idx[1]]
        u = (np.random.uniform(0, 1, size=(len(ind.genes)) ) <= cr)
        u[np.random.choice(len(ind.genes))] = True
        new_genes = np.where(u, 
            pbest.genes + f * (ind_ran1.genes - ind_ran2.genes),
            ind.genes
        )
        bounds = ind.bounds
        for j in range(len(new_genes)):
                if new_genes[j] > bounds[1]:
                    new_genes[j] = (bounds[1] + ind.genes[j]) / 2
                elif new_genes[j] < bounds[0]:
                    new_genes[j] = (bounds[0] + ind.genes[j]) / 2

        new_ind = Individual(ind.MAX_NVARS, bounds)
        new_ind.genes = new_genes
        new_ind.cal_fitness(self.prob)
        return new_ind
    def split(self, pool):
        pop_list = list(len(pool) * np.array(self.ratio))
        pop_list = [int(x) for x in pop_list]
        max_idx = np.argmax(pop_list)
        min_idx = np.argmin(pop_list)
        if sum(pop_list) > len(pool):
            pop_list[max_idx] -= (sum(pop_list) - len(pool))
        if sum(pop_list) < len(pool):
            pop_list[min_idx] += (len(pool) - sum(pop_list))
        max_idx = np.argmax(pop_list)
        min_idx = np.argmin(pop_list)
        # print(pop_list)
        assert sum(pop_list) == len(pool), f"Sum of pop_list: {sum(pop_list)} != len(pool): {len(pool)}"
        while 0 in pop_list:
            zero_idx = pop_list.index(0)
            if max_idx != zero_idx:
                pop_list[zero_idx] += 1
                pop_list[max_idx] -= 1
            else:
                raise ValueError("Cannot split pool: all ratios are 0")
        
        assert sum(pop_list) == len(pool), f"Sum of pop_list: {sum(pop_list)} != len(pool): {len(pool)}"
        Pop_list = []
        start = 0
        for size in pop_list:
            end = start + size
            Pop_list.append(pool[start: end])
            start = end
        # assert sum(pop_list) == len(pool), f"Sum of pop_list: {sum(pop_list)} != len(pool): {len(pool)}"
        return Pop_list
    def addarchive(self, ind, pop_size):
        caution_size = int(self.arc_rate * pop_size)
        while len(self.archive) > caution_size:
            r = random.randint(0, len(self.archive) - 1)
            self.archive.pop(r)
        self.archive.append(ind)

    def search(self, pool):
        pool.sort(key = lambda ind: ind.fitness)
        best = pool[: max( int(0.1*len(pool)), 2 )]
        new_pool = []
        operators = [self.cur_topbest, self.cur_topbestav, self.pbest_ind]
        quality = [0]*len(operators)
        div = [0]*len(operators)
        improv_rate = [0]*len(operators)
        accumulate_diff = 0
        old_FEs = self.prob.FE
        random.shuffle(pool)
        pop_list = self.split(pool)
        # print(pop_list)
        for idx, pop in enumerate(pop_list):
            for ind in pop:
                r = random.randint(0, self.H - 1)
                f, cr = self.generateFCR(self.mem_f[r], self.mem_cr[r])
                new_ind = operators[idx](pool, ind, cr, f, best)
                if ind.fitness > new_ind.fitness:
                    self.diff_f.append(ind.fitness - new_ind.fitness)
                    self.s_cr.append(cr)
                    self.s_f.append(f)
                    self.addarchive(ind, len(pool))
                    new_pool.append(new_ind)
                    accumulate_diff += ind.fitness- new_ind.fitness

                else:
                    new_pool.append(ind)
            # get information of operators
            pop.sort(key = lambda ind: ind.fitness)
            quality[idx] = pop[0].fitness
            div[idx] = np.sum([np.linalg.norm(ind.genes - pop[0].genes) for ind in pop])
        qual_sum = sum(quality)
        div_sum = sum(div)
        for i in range(len(quality)):
            quality[i] = quality[i]/qual_sum if qual_sum != 0 else 0 
            div[i] = div[i]/div_sum if div_sum != 0 else 0
        for i in range(len(operators)):
            improv_rate[i] = (1 - quality[i]) + div[i]
        sum_improv_rate = sum(improv_rate)
        if sum_improv_rate == 0:
            self.ratio = [1/3 , 1/3, 1/3]
        else:
            self.ratio = [max(0.1, min(0.9, x/sum_improv_rate)) for x in improv_rate]
        # print(self.ratio)
        min_fitness1 = min(ind.fitness for ind in pool)
        min_fitness2 = min(ind.fitness for ind in new_pool)
        assert min_fitness2 <= min_fitness1, f'Fitness of new pool: {min_fitness2} > old pool:{min_fitness1} ???'
        new_fes = self.prob.FE
        self.effective = accumulate_diff / (new_fes - old_FEs)
        self.updateMemory()
        return new_pool
    def updateMemoryCR(self, diff_f, s_cr):
        diff_f = np.array(diff_f)
        s_cr = np.array(s_cr)

        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tu = sum(weight * s_cr * s_cr)
        mau = sum(weight * s_cr)
        return tu/mau
    def updateMemoryF(self, diff_f, s_f):
        diff_f = np.array(diff_f)
        s_f = np.array(s_f)

        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tu = sum(weight * s_f * s_f)
        mau = sum(weight * s_f)
        return tu/mau
    def updateMemory(self):
        if len(self.s_cr) > 0:
            if self.mem_cr[self.mem_pos] == "terminal" or max(self.s_cr)==0:
                self.mem_cr[self.mem_pos] = "terminal"
            else:
                self.mem_cr[self.mem_pos] = self.updateMemoryCR(self.diff_f, self.s_cr)
            self.mem_f[self.mem_pos] = self.updateMemoryF(self.diff_f, self.s_f)
            self.mem_pos = (self.mem_pos + 1) % self.H
            self.s_cr = []
            self.s_f = []
            self.diff_f = []
    
        else:
            # do nothing
            pass

In [30]:
class MTSLS1:
    def __init__(self, prob):
        self.prob = prob
        self.SR = np.array([0.2]*30)
        self.effective = float('inf')
    def __mtsls_improve_dim(self, sol, i, SR):
        best_fitness = sol.fitness
        newsol = copy.deepcopy(sol)
        newsol.genes[i] -= ((sol.bounds[1] - sol.bounds[0])*SR[i] + sol.bounds[0])
        newsol.genes = np.clip(newsol.genes, sol.bounds[0], sol.bounds[1])
        newsol.cal_fitness(self.prob)
        fitness_newsol = newsol.fitness
        if fitness_newsol < best_fitness:
            sol = newsol 
        elif fitness_newsol > best_fitness:
            newsol = copy.deepcopy(sol)
            newsol.genes[i] += (200*0.5*SR[i] - 100)
            newsol.genes = np.clip(newsol.genes, -100,100)
            newsol.cal_fitness(self.prob)
            fitness_newsol = newsol.fitness
            if fitness_newsol < best_fitness:
                sol = newsol
        return sol
    def search(self, ind, maxeval):
        dim = len(ind.genes)
        improvement = np.zeros(dim)
        dim_sorted = np.random.permutation(dim)
        improved_dim = np.zeros(dim)
        current_best = copy.deepcopy(ind)
        before_fitness = current_best.fitness
        maxevals = self.prob.FE + maxeval
        if self.prob.FE < maxevals:
            dim_sorted = np.random.permutation(dim)

            for i in dim_sorted:
                result = self.__mtsls_improve_dim(current_best, i, self.SR)
                improve = max(current_best.fitness - result.fitness, 0)
                improvement[i] = improve
                if improve:
                    improved_dim[i] = 1
                    current_best = result
                else:
                    self.SR[i] /=2
            dim_sorted = improvement.argsort()[::-1]
            d = 0
        while self.prob.FE < maxevals:
            i = dim_sorted[d]
            result = self.__mtsls_improve_dim(current_best, i, self.SR)
            improve = max(current_best.fitness- result.fitness, 0)
            improvement[i] = improve
            next_d = (d+1)%dim 
            next_i = dim_sorted[next_d]
            if improve:
                improved_dim[i] = 1
                current_best = result
                if improvement[i] < improvement[next_i]:
                    dim_sorted = improvement.argsort()[::-1]
            else:
                self.SR[i] /= 2
                d = next_d
        initial_SR = 0.2 
        self.SR[self.SR < 1e-15] = initial_SR
        after_fitness = current_best.fitness

        if after_fitness > before_fitness:
            raise ValueError(f"MTSLS1-Error: Fitness must be improved, but got {after_fitness - before_fitness}", )
        self.effective = (before_fitness - after_fitness) / abs(before_fitness)
        return current_best
    def reset(self):
        self.SR = np.array([0.5]*30)

class LBFGSB:
    def __init__(self, prob):
        self.prob = prob
        self.effective = float('inf')
    def search(self, ind, maxeval):

        before_fitness = ind.fitness
        sol, fit, info = fmin_l_bfgs_b(self.prob.fitness_of_ind, ind.genes, approx_grad=True, bounds=[(-100,100)]*len(ind.genes), maxfun=maxeval)
        # sol2, fit2, info2 = fmin_l_bfgs_b(self.prob[ind.skill_factor-1], np.random.rand(len(ind.genes)), approx_grad=True, bounds=[(0,1)]*len(ind.genes), maxfun=maxeval)
        if fit > before_fitness:
            new_ind = ind
        else:
            new_ind = copy.deepcopy(ind)
            new_ind.genes = sol
            new_ind.fitness = fit
        after_fitness = new_ind.fitness
        if after_fitness > before_fitness:
            raise ValueError(f"BFGS-Error: Fitness must be improved, but got {after_fitness - before_fitness}", )
        self.effective = (before_fitness - after_fitness) / abs(before_fitness)

        return new_ind
    def reset(self):
        pass
class LBFGSB2:
    def __init__(self, prob):
        self.prob = prob
        self.effective = float('inf')
        self.stop = False
        self.grad = None
    def search(self, ind, maxeval):
        before_fitness = ind.fitness
        if self.stop:
            # if LBFGSB doesn't improve the fitness, we pass a random individual to algo
            input_genes = np.random.uniform(-100, 100, len(ind.genes))
        else:
            input_genes = ind.genes
        sol, fit, info = fmin_l_bfgs_b(self.prob.fitness_of_ind, input_genes, approx_grad=True, bounds=[(-100,100)]*len(ind.genes), maxfun=maxeval)
        #print(f'FITNESS OF IND: {fit}, RANDOM GENES: {self.stop}')
        self.grad = info["grad"]
        if fit > before_fitness:
            new_ind = ind
        else:
            new_ind = copy.deepcopy(ind)
            new_ind.genes = sol
            new_ind.fitness = fit
        after_fitness = new_ind.fitness
        if after_fitness > before_fitness:
            raise ValueError(f"LBFGSB2-Error: Fitness must be improved, but got {after_fitness - before_fitness}", )
        self.effective = (before_fitness - after_fitness) / abs(before_fitness)
        if self.effective == 0:
            self.stop = True

        return new_ind


In [31]:
import numpy as np
import copy, math

class Salp(object):
    def __init__(self):
        self.X = []
        self.fitness = np.inf
    
    # def __repr__(self):
    #     return str(list(self.X))
    
    def getX(self):
        return self.X
    
    def setX(self , X):
        self.X = X

    def get_fitness(self):
        return self.fitness
    
    def set_fitness(self, f):
        self.fitness = f

class ASSA(object):
    def __init__(self, n = 50, max_iters = 10000, seed = None, former_leader=None, verbose = True, opt_value = 0, maxFes = 500000,
                 nmin = 20, dim = 30, lb = -1, ub = 1):
        if seed is not None:
            np.random.seed(seed)
        self.dim = dim
        self.former_leader = former_leader
        self.lb = lb
        self.ub = ub
        self._F = None
        self.c1 = 1
        self.n = n
        self.nmax = n 
        self.nmin = nmin
        self.max_iters = max_iters
        self.iteration = 1
        self.Salps = []
        self.verbose = verbose
        self.opt_value = opt_value
        self.FEs = 0
        self.maxFEs = maxFes
        self.funct = None

    def _initialise(self):
        return np.random.uniform(0,1,self.dim)*(self.ub - self.lb) + self.lb
    def update_fitness(self):
        try:
            for s in self.Salps[:self.n]:
                fitness = self.funct(s.getX())
                s.set_fitness(fitness)
        except:
            print("* Error! The function", self.funct, "is not defined. Please, provide a valid function.")
            exit(-1)
    
    def _create_salps(self):
        # we can add a individual from IMODE to this population pool
        if self.former_leader is not None:
            s = Salp()
            s.setX(self.former_leader)
            self.Salps.append(s)
            for _ in range(self.n-1):
                s = Salp()
                s.setX(self._initialise())
                self.Salps.append(s)
        else:
            for _ in range(self.n):
                s = Salp()
                s.setX(self._initialise())
                self.Salps.append(s)
        
        self.update_fitness()
        self.FEs = self.FEs + self.n

        self.Salps = sorted(self.Salps, key= lambda x: x.get_fitness(), reverse= False)

        self._F = copy.deepcopy(self.Salps[0])

        if self.verbose:
            print(f'* {self.n} salps have been created')
            print(f'* Iter {self.iteration}: best fitness {self._F.get_fitness()}')

    def _termination_criterion(self):
        if self.FEs > self.maxFEs-self.nmin:
            if self.verbose:
                print(f"The number of FEs has been used: {self.FEs}")
            return True
        else:
            return False
        
    def _updateC1(self):
        # tmp = np.random.rand() + (10*self.iteration)/self.max_iters
        # self.c1 = self.cmax + (self.cmin - self.cmax)*math.log10(tmp)

        self.c1 = 2 * math.exp(-((4 * self.iteration / self.max_iters) ** 2))

    def _updateN(self):
        tmp = ((self.nmin - self.nmax)/self.maxFEs)*self.FEs + self.nmax
        self.n = int(tmp)
    
    def _update_leader(self, index):
        c2 = np.random.uniform(0, 1, self.dim)
        c3 = np.random.uniform(0, 1, self.dim)

        F = copy.deepcopy(self._F.getX())
        X = np.zeros(self.dim)

        tmp2 = self.c1*((self.ub - self.lb)*c2 + self.lb)
        X[c3<0.5] = F[c3<0.5] + tmp2[c3<0.5]
        X[c3>=0.5] = F[c3>=0.5] - tmp2[c3>=0.5]

        self.Salps[index].setX(X)

    def _update_salps(self):
        for i in range(self.n):
            if (i<self.n /2):
                self._update_leader(i)

            else:
                X1= copy.deepcopy(self.Salps[i].getX())
                X2 = copy.deepcopy(self.Salps[i-1].getX())
                X = 0.5 *(X1+X2)
                self.Salps[i].setX(X)

        for s in self.Salps[:self.n]:
            X = copy.deepcopy(s.getX())
            for i in range(self.dim):
                X[i] = np.clip(X[i], self.lb, self.ub)
            
            s.setX(X)

    def _iterate(self):
        self._updateC1()
        self._updateN()

        self._update_salps()

        self.update_fitness()
        self.FEs = self.FEs + self.n

        for s in self.Salps:
            if s.get_fitness() < self._F.get_fitness():
                self._F = copy.deepcopy(s)

        if(self.verbose) & (self.iteration%100 == 0):
            print(f'* Iteration {self.iteration}: best fitness {self._F.get_fitness()}--- var {self._F.get_fitness()- self.opt_value}')
        
        self.iteration += 1

    def optimize(self, function):  # Return best position and best value
        self.funct = function
        if self.verbose:
            print("Starting ASSA code...")
        
        self._create_salps()

        while not self._termination_criterion():
            self._iterate()

        if self.verbose:
            print(f'Process terminated')

        return self._F.getX(), self._F.get_fitness()

In [32]:
class MSHO:
    def __init__(self, prob, gen_length, new_LS=True,BASE_POPSZ=100, BASE_rmp=0.3, update_rate = 0.06, learning=True, dynamic_pop=True, phasethree=False):
        self.gnbg = prob
        self.prob = prob
        self.base_popsize = BASE_POPSZ
        self.max_popsize = BASE_POPSZ
        self.min_popsize = int(BASE_POPSZ/5)
        self.inds_tasks = self.base_popsize
        self.base_rmp = BASE_rmp
        self.gen_length = gen_length
        self.delta = None 
        self.s_rmp = None
        self.update_rate = update_rate
        if new_LS:
            self.learningPhase = LearningPhaseILSVer2(self.prob)
        else:
            self.learningPhase = LearningPhaseILS(self.prob)
        self.learningPhase = LearningPhaseILS(self.prob)
        self.learningPhase2 = LearningPhase(is_start = True, prob=self.prob) 
        self.learning = learning
        self.dynamic_pop = dynamic_pop
        # self.log = [ [] for _ in range(len(prob))]
        # self.rmp_report = []
        # self.bmb = []
        self.phasethree = phasethree
        self.mutate = Mutation(self.prob)
        # self.das_crossover = das_crossover
    def run(self, checkpoint = None):
        accept = -1
        self.prob.FE = 0
        best = []
        if checkpoint is None:
            pop = Population(self.inds_tasks, self.gen_length, self.prob)
            pop.init()
        if checkpoint is not None:
            pop = checkpoint 
        
        optimized = False
        while True:
            if self.prob.FE > self.prob.max_FE:
                pop.pop.sort(key = lambda ind: ind.fitness)
                print(pop.pop[0].genes)
                break
            this_pop_result = pop.get_result()
            print(f'At: {self.prob.FE}, found best fitness: {this_pop_result}, var : {this_pop_result - self.prob.opt_value}')
            old_fes = self.prob.FE
            # 1. reproduction population
            offsprings = self.reproduction(pop, self.inds_tasks)
            pop.pop.extend(offsprings)
            # 2. decrease pop size
            if self.dynamic_pop:
                self.inds_tasks = int(
                    int(max((self.min_popsize - self.max_popsize) * (self.prob.FE/self.prob.max_FE) + self.max_popsize, self.min_popsize))
                )
            pop.selection_EMEBI(self.inds_tasks) 
            # 3. Employ phaseThree(like phase2 in EME-BI) 
            # and phaseTwo(local search the best individual)

            if self.learning:
                pop = self.phaseThree(pop)
            if self.phasethree:
                pop = self.phaseTwo(pop)
            new_fes = self.prob.FE
            
            # 4. If satisfy some condition, then use ASSA
            if self.prob.FE > 500000:
                pop.pop.sort(key = lambda ind: ind.fitness)
                assa = ASSA(n = 50, max_iters = 10000,maxFes= 600000, opt_value = self.prob.opt_value,nmin = 20, dim = self.prob.dim, lb = self.prob.LB, ub=self.prob.UB)
                x,y = assa.optimize(self.prob.fitness_of_ind)
            
            return f'{self.prob.best_val} {self.prob.FE}'
        

    



            

    def reproduction(self, pop, SIZE):
        offs = []
        population = pop.pop
        terminateCondition = False
        counter = 0
        while not terminateCondition:
            idx_1, idx_2 = np.random.choice(len(population), 2)
            ind1 = population[idx_1]
            ind2 = population[idx_2]
            off1, off2 = pop.crossover(ind1, ind2)
            off1.cal_fitness(self.prob)
            off2.cal_fitness(self.prob)
            offs.extend([off1, off2])
            counter += 2
            terminateCondition = (counter >= SIZE)
        return offs

    def phaseTwo(self, pop):
        newPop = []
        # evolve(pop, DE_evals , LS_evals)
        nextPop = self.learningPhase.evolve(pop.pop, DE_evals=1000, LS_evals=100)
        newPop.extend(nextPop)
        pop.pop = newPop
        return pop
    
    def phaseThree(self, pop):
        newPop = []
        nextPop = self.learningPhase2.evolve(pop.pop)
        newPop.extend(nextPop)
        pop.pop = newPop 
        return pop


In [33]:
# new_LS = True , phasethree=True --> dung local search moi cai tien
# new_LS = False , phasethree = True --> dung local search cu
# phasethree = True --> bat local search
# phasethree = False --> tat local search
# tat standby phase --> comment SSA 
for prob_idx in range(1, 25):
    algo = MSHO( prob=GNBG(prob_idx), gen_length=30, new_LS=True, phasethree=True)
    res = algo.run()
    with open(f'f{prob_idx}.txt', 'w') as f:
        f.write(str(res))

FileNotFoundError: [Errno 2] No such file or directory: './gnbg_func/f1.mat'